In [1]:
import warnings
warnings.filterwarnings("ignore")
import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import math
import matplotlib.pyplot as plt
import contractions
import re
import random
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
additional  = ['rt','rts','retweet'] # lista adicional de stopwords
swords = set().union(stopwords.words('english'),additional) # adicionando palavras para o stopwords

pd.options.display.float_format = '{:,.2f}'.format

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rbsa2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# controla tamanho geral das figuras no matplotlib/seaborn
TAMANHO_GERAL = [15, 10]
plt.rcParams['figure.figsize'] = TAMANHO_GERAL
sns.set(rc={'figure.figsize' : TAMANHO_GERAL})

In [3]:
# https://www.kaggle.com/kaushiksuresh147/bitcoin-tweets
p = 0.005
p = 1

dataFrame = pd.read_csv('D:\SiDi\Project\dataset\Bitcoin_tweets.csv', skiprows=lambda i: i>0 and random.random() > p, lineterminator='\n' )
# dataFrame['date'] = pd.to_datetime(dataFrame['date']).dt.date
# twFrame = pd.read_csv('Data/Bitcoin_tweets_6_2_2021.csv')dataFrame 
#dataFrame = pd.read_csv('tweets.csv', delimiter=';', skiprows=0, lineterminator='\n' )
dataFrame.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,"8,534.00",7605,4838,False,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],Twitter Web App,False
1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,"6,769.00",1532,25483,False,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",Twitter for Android,False
2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.00,332,924,False,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",NaN,Twitter Web App,False
3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.00,129,14,False,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",dlvr.it,False
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,"1,249.00",1472,10482,False,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,['BTC'],Twitter Web App,False


In [13]:
dataFrame = dataFrame.dropna(subset=['date'])

In [14]:
indexes2Drop = dataFrame[dataFrame['date'].str.contains("bitcoin|Bitcoin|ETH|btc|BTC")].index
dataFrame.drop(indexes2Drop,inplace=True)

In [15]:
dataFrame['date'] = pd.to_datetime(dataFrame['date']).dt.date
# pd.to_datetime(dataFrame['date'])dt.date

In [16]:
twFrame = dataFrame[['date','text']]
twFrame

,date,text
0,2021-02-10,Blue Ridge Bank shares halted by NYSE after #b...
1,2021-02-10,"😎 Today, that's this #Thursday, we will do a ""..."
2,2021-02-10,"Guys evening, I have read this article about B..."
3,2021-02-10,$BTC A big chance in a billion! Price: \487264...
4,2021-02-10,This network is secured by 9 508 nodes as of t...
...,...,...
1905880,2021-11-05,@shibarium_ BTC is the cutest thing in the wor...
1905881,2021-11-05,@TechDev_52 @JohalMiles Will give him a follow...
1905882,2021-11-05,@CryptoSagaing I have been buying cryptocurren...
1905883,2021-11-05,NGL wouldn’t mind another little quick dip dow...


In [17]:
twText = twFrame[['text']]
twText

,text
0,Blue Ridge Bank shares halted by NYSE after #b...
1,"😎 Today, that's this #Thursday, we will do a ""..."
2,"Guys evening, I have read this article about B..."
3,$BTC A big chance in a billion! Price: \487264...
4,This network is secured by 9 508 nodes as of t...
...,...
1905880,@shibarium_ BTC is the cutest thing in the wor...
1905881,@TechDev_52 @JohalMiles Will give him a follow...
1905882,@CryptoSagaing I have been buying cryptocurren...
1905883,NGL wouldn’t mind another little quick dip dow...


In [18]:
twText['text'] = twText['text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0]) # remove urls
twText['text'] = twText['text'].str.replace('\d+', '') # remove valores numericos
twText.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
text = twText['text'][0]
text

'Blue Ridge Bank shares halted by NYSE after #bitcoin ATM announcement '

In [19]:
def tweet_to_words(tweet):
    ''' Convert tweet text into a sequence of words '''
    
    # convert to lowercase
    text = tweet.lower()
    # remove non letters
    text = re.sub("@[A-Za-z0-9_]+","", text) #remove menções
    text = re.sub(r'#[a-zA-z0-9]+','',text) # remove hashtag
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)        
    text = re.sub('[()!?]', ' ', text)
    text = re.sub('\[.*?\]',' ', text)
    text = re.sub("[^a-z0-9]"," ", text)
    text = contractions.fix(text)
    # text = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI)
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    # tokenize
    words = text.split()
    # remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    # apply stemming
    # words = [PorterStemmer().stem(w) for w in words]
    # return list
    return words

In [20]:
twFrame['cleantext']=twText['text'].map(lambda x: tweet_to_words(x))
twFrame

,date,text,cleantext
0,2021-02-10,Blue Ridge Bank shares halted by NYSE after #b...,"[blue, ridge, bank, shares, halted, nyse, atm,..."
1,2021-02-10,"😎 Today, that's this #Thursday, we will do a ""...","[today, take, friend, expe]"
2,2021-02-10,"Guys evening, I have read this article about B...","[guys, evening, read, article, btc, would, lik..."
3,2021-02-10,$BTC A big chance in a billion! Price: \487264...,"[btc, big, chance, billion, price]"
4,2021-02-10,This network is secured by 9 508 nodes as of t...,"[network, secured, nodes, today, soon, biggest..."
...,...,...,...
1905880,2021-11-05,@shibarium_ BTC is the cutest thing in the wor...,"[btc, cutest, thing, world, many, coins, chang..."
1905881,2021-11-05,@TechDev_52 @JohalMiles Will give him a follow...,"[give, follow]"
1905882,2021-11-05,@CryptoSagaing I have been buying cryptocurren...,"[buying, cryptocurrencies, since, last, year, ..."
1905883,2021-11-05,NGL wouldn’t mind another little quick dip dow...,"[ngl, mind, another, little, quick, dip, blok]"


In [21]:
pd.DatetimeIndex(twFrame["date"]).date

array([datetime.date(2021, 2, 10), datetime.date(2021, 2, 10),
       datetime.date(2021, 2, 10), ..., datetime.date(2021, 11, 5),
       datetime.date(2021, 11, 5), datetime.date(2021, 11, 5)],
      dtype=object)

In [22]:
# import contractions
# text = '''She'd like to know how I'd done that! 
#           She's going to the park and I don't think I'll be home for dinner.
#           Theyre going to the zoo and she'll be home for dinner.'''
  
# text = contractions.fix(text)
# print(text)

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# import SentimentIntensityAnalyzer class
# from vaderSentiment.vaderSentiment module.
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
 
# function to print sentiments
# of the sentence.
def sentiment_scores(sentence):
 
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
 
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)
    
    return sentiment_dict['compound']
     
 
    # decide sentiment as positive, negative and neutral
#     if sentiment_dict['compound'] >= 0.05 :
#         return "Positive"
 
#     elif sentiment_dict['compound'] <= - 0.05 :
#          return  "Negative"
 
#     else :
#          return "Neutral"

In [ ]:
%%time
sentimental = [sentiment_scores(i) for i in twFrame['cleantext']]

twFrame["Sentimento"] = sentimental
twFrame["Sentimento"]

In [ ]:
twFrame["Sentimento"].describe()

In [ ]:
pd.DatetimeIndex(twFrame['date']).normalize()
# twFrame.groupby(pd.DatetimeIndex(twFrame['date']).normalize())['Sentimento'].sum().sort_values()
twFrame.groupby(pd.DatetimeIndex(twFrame['date']).normalize())['Sentimento'].sum().plot(legend=True)


In [ ]:
# twFrame[(twFrame["Sentimento"].sort_values() > 0.2) | (twFrame["Sentimento"].sort_values() < -0.2) ]
# twFrame.groupby(pd.DatetimeIndex(twFrame['date']).normalize())['Sentimento'].sum()
g = twFrame.groupby(pd.DatetimeIndex(twFrame['date']).normalize()).sum()
g.reset_index(inplace = True)
g

In [ ]:

minmaxpoints = g[ (g['Sentimento'] >= 0.2) | (g['Sentimento'] <= -0.2) ]
minmaxpoints['date'] = pd.to_datetime(minmaxpoints['date']).dt.date
# minmaxpoints['date'] = pd.to_datetime(minmaxpoints['date'], format='%y-%m-%d')
minmaxpoints

In [ ]:
minmaxpoints['date'] = minmaxpoints['date'].astype(str)
aa = list(minmaxpoints['date'])
aa

In [ ]:
twFrame.to_csv('D:\\SiDi\\Project\\dataset\\tweet_sentiment_analysis.csv')

In [ ]:
btcPrice = pd.read_csv('D:\\SiDi\\Project\\dataset\\BTC-USD.csv')
# btcPrice[btcPrice['Date'] == '2021-02-08']
btcPrice

In [ ]:
btcPrice.columns

In [ ]:
#teste

In [ ]:
# btcPrice.loc[btcPrice['Date'] == teste]
# btcPrice[btcPrice['Date'] == teste]
    

In [ ]:
# btcPrice.loc[btcPrice['Date'].isin([aa])]
zz = btcPrice.query('Date in @aa')
zz
btcPrice.loc[zz.index,'teste'] = True
btcPrice['teste'] = btcPrice['teste'].replace(np.nan,False)
btcPrice['Date']

In [ ]:
# btcPrice.plot(x='Date', y=['Close','teste'])


# fig, ax1 = plt.subplots()
# ax1.plot(btcPrice['Date'],btcPrice['Close'])
# ax1.plot(btcPrice['Date'],btcPrice['teste'],marker='o')

ts = btcPrice['Close'] 
ts2 = ts[btcPrice['teste']]

fig, ax1 = plt.subplots()
ax1.plot(btcPrice['Date'], ts)       
ax1.plot(ts2.index, ts2,'g^')
fig.autofmt_xdate()